In [1]:
import math
import numpy
from gnuradio import gr, digital
from gnuradio import analog
from gnuradio import blocks
import sys
from ITpp import BCH_Encoder, BCH_Decoder


try:
    from scipy.special import comb
    from scipy.special import erfc
except ImportError:
    print "Error: could not import scipy (http://www.scipy.org/)"
    sys.exit(1)

try:
    import pylab
except ImportError:
    print "Error: could not import pylab (http://matplotlib.sourceforge.net/)"
    sys.exit(1)

# Best to choose powers of 10
#N_BITS = 1e7
N_BITS = 1e7
RAND_SEED = 42

In [2]:
def Q(x):
    return (0.5 * erfc(x/(numpy.sqrt(2))))

def Pb_BPSK(EbN0):
    return Q(numpy.sqrt(2*EbN0))

def Pb_DBPSK(EbN0):
    return 2 * Q(numpy.sqrt(2*EbN0)) - 2 * Q(numpy.sqrt(2*EbN0))**2

def from_dB(x):
    return math.pow(10, x/10)

def Pb_Hard_Codes(p, N, t):
    Sum = 0
    for m in range(t+1, N + 1):
        x = (m + t) * comb(N, m, exact = True, repetition = False) * p ** (m) * (1 - p) ** (N - m)
        Sum += x
    return Sum/N

def berawgn_BPSK(EbN0):
    """ Calculates theoretical bit error rate in AWGN (for BPSK and given Eb/N0) """
    return Pb_BPSK(from_dB(EbN0))

def berawgn_DBPSK(EbN0):
    """ Calculates theoretical bit error rate in AWGN (for DBPSK and given Eb/N0) """
    return Pb_DBPSK(from_dB(EbN0))


class BitErrors(gr.hier_block2):
    """ Two inputs: true and received bits. We compare them and
    add up the number of incorrect bits. Because integrate_ff()
    can only add up a certain number of values, the output is
    not a scalar, but a sequence of values, the sum of which is
    the BER. """
    def __init__(self, bits_per_byte):
        gr.hier_block2.__init__(self, "BitErrors",
                gr.io_signature(2, 2, gr.sizeof_char),
                gr.io_signature(1, 1, gr.sizeof_int))

        # Bit comparison
        comp = blocks.xor_bb()
        intdump_decim = 100000

	#print(bits_per_byte)

        if N_BITS < intdump_decim:
            intdump_decim = int(N_BITS)
        self.connect(self,
                     comp,
                     blocks.unpack_k_bits_bb(bits_per_byte),
                     blocks.uchar_to_float(),
                     blocks.integrate_ff(intdump_decim),
                     blocks.multiply_const_ff(1.0/N_BITS),
                     self)
        self.connect((self, 1), (comp, 1))

In [3]:
class BERAWGNSimu(gr.top_block):
    " This contains the simulation flow graph "
    def __init__(self, EbN0):
        gr.top_block.__init__(self)
        self.const = digital.bpsk_constellation()
        # Source is N_BITS bits, non-repeated

	#print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
	#print("arity: ", self.const.arity())

        data = map(int, numpy.random.randint(0, self.const.arity(), int(N_BITS/self.const.bits_per_symbol())))
        src   = blocks.vector_source_b(data, False)
        mod   = digital.chunks_to_symbols_bc((self.const.points()), 1)
        add   = blocks.add_vcc()
# 	diff_enc = digital.diff_encoder_bb(2)
# 	diff_dec = digital.diff_decoder_bb(2)

	#print("Gaussian amplitude: ", self.EbN0_to_noise_voltage(EbN0))

        noise = analog.noise_source_c(analog.GR_GAUSSIAN,
                                      self.EbN0_to_noise_voltage(EbN0),
                                      RAND_SEED)
        demod = digital.constellation_decoder_cb(self.const.base())
        ber   = BitErrors(self.const.bits_per_symbol())
        self.sink  = blocks.vector_sink_f()
        self.connect(src, mod, add, demod, ber, self.sink)
        self.connect(noise, (add, 1))
        self.connect(src, (ber, 1))

    def EbN0_to_noise_voltage(self, EbN0):
        """ Converts Eb/N0 to a complex noise voltage (assuming unit symbol power) """
	#print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        return 1.0 / math.sqrt(self.const.bits_per_symbol() * 10**(float(EbN0)/10))

In [4]:
def simulate_ber(EbN0):
    """ All the work's done here: create flow graph, run, read out BER """
    print "Eb/N0 = {} dB".format(EbN0)
    fg = BERAWGNSimu(EbN0)
    fg.run()
    res = numpy.sum(fg.sink.data())
    print("BER: {}\n".format(res))
    return res

In [5]:
EbN0_min = 0
EbN0_max = 10
EbN0_range = range(EbN0_min, EbN0_max+1)
#EbN0_range = [0.5*x for x in range(EbN0_min*2, (EbN0_max+1)*2)]

In [6]:
ber_simu = [simulate_ber(x) for x in EbN0_range]

Eb/N0 = 0 dB
BER: 0.078780001204

Eb/N0 = 1 dB
BER: 0.0563720004284

Eb/N0 = 2 dB
BER: 0.0375418004696

Eb/N0 = 3 dB
BER: 0.0229000002873

Eb/N0 = 4 dB
BER: 0.0125480001443

Eb/N0 = 5 dB
BER: 0.00598980008363

Eb/N0 = 6 dB
BER: 0.00238320003518

Eb/N0 = 7 dB
BER: 0.000777000011112

Eb/N0 = 8 dB
BER: 0.00019040000177

Eb/N0 = 9 dB
BER: 3.43000006211e-05

Eb/N0 = 10 dB
BER: 3.60000004207e-06



In [7]:
class BERAWGNSimu2(gr.top_block):
    " This contains the simulation flow graph "
    def __init__(self, EbN0):
        gr.top_block.__init__(self)
        self.const = digital.bpsk_constellation()
        # Source is N_BITS bits, non-repeated

        #print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        #print("arity: ", self.const.arity())

        data = map(int, numpy.random.randint(0, self.const.arity(), int(N_BITS/self.const.bits_per_symbol())))
        src   = blocks.vector_source_b(data, False)
        enc   = BCH_Encoder(15, 2)
        mod   = digital.chunks_to_symbols_bc((self.const.points()), 1)
        add   = blocks.add_vcc()

        #print("Gaussian amplitude: ", self.EbN0_to_noise_voltage(EbN0))

        noise = analog.noise_source_c(analog.GR_GAUSSIAN,
                                      self.EbN0_to_noise_voltage(EbN0),
                                      RAND_SEED)
        demod = digital.constellation_decoder_cb(self.const.base())
        dec   = BCH_Decoder(15, 2)
        ber   = BitErrors(self.const.bits_per_symbol())
        self.sink  = blocks.vector_sink_f()
        self.connect(src, enc, mod, add, demod, dec, ber, self.sink)
        self.connect(noise, (add, 1))
        self.connect(src, (ber, 1))

    def EbN0_to_noise_voltage(self, EbN0):
        """ Converts Eb/N0 to a complex noise voltage (assuming unit symbol power) """
        #print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        return 1.0 / math.sqrt(self.const.bits_per_symbol() * 10**(float(EbN0-10*math.log10(15.0/7.0))/10))

In [8]:
def simulate_ber2(EbN0):
    """ All the work's done here: create flow graph, run, read out BER """
    print "Eb/N0 = {} dB".format(EbN0)
    fg = BERAWGNSimu2(EbN0)
    fg.run()
    res = numpy.sum(fg.sink.data())
    print("BER: {}\n".format(res))
    return res

In [9]:
ber_simu2  = [simulate_ber2(x) for x in EbN0_range]

Eb/N0 = 0 dB
BER: 0.210722803837

Eb/N0 = 1 dB
BER: 0.155745301745

Eb/N0 = 2 dB
BER: 0.103413700592

Eb/N0 = 3 dB
BER: 0.0591377004748

Eb/N0 = 4 dB
BER: 0.0283523003163

Eb/N0 = 5 dB
BER: 0.0107164001165

Eb/N0 = 6 dB
BER: 0.00301170002422

Eb/N0 = 7 dB
BER: 0.000633700004983

Eb/N0 = 8 dB
BER: 7.39000007002e-05

Eb/N0 = 9 dB
BER: 6.00000008433e-06

Eb/N0 = 10 dB
BER: 6.00000021223e-07



In [10]:
class BERAWGNSimu3(gr.top_block):
    " This contains the simulation flow graph "
    def __init__(self, EbN0):
        gr.top_block.__init__(self)
        self.const = digital.bpsk_constellation()
        # Source is N_BITS bits, non-repeated

        #print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        #print("arity: ", self.const.arity())

        data = map(int, numpy.random.randint(0, self.const.arity(), int(N_BITS/self.const.bits_per_symbol())))
        src   = blocks.vector_source_b(data, False)
        enc   = BCH_Encoder(15, 2)
        mod   = digital.chunks_to_symbols_bc((self.const.points()), 1)
        add   = blocks.add_vcc()
        diff_enc = digital.diff_encoder_bb(2)
        diff_dec = digital.diff_decoder_bb(2)

        #print("Gaussian amplitude: ", self.EbN0_to_noise_voltage(EbN0))

        noise = analog.noise_source_c(analog.GR_GAUSSIAN,
                                      self.EbN0_to_noise_voltage(EbN0),
                                      RAND_SEED)
        demod = digital.constellation_decoder_cb(self.const.base())
        dec   = BCH_Decoder(15, 2)
        ber   = BitErrors(self.const.bits_per_symbol())
        self.sink  = blocks.vector_sink_f()
        self.connect(src, diff_enc, enc, mod, add, demod, dec, diff_dec, ber, self.sink)
        self.connect(noise, (add, 1))
        self.connect(src, (ber, 1))

    def EbN0_to_noise_voltage(self, EbN0):
        """ Converts Eb/N0 to a complex noise voltage (assuming unit symbol power) """
        #print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        return 1.0 / math.sqrt(self.const.bits_per_symbol() * 10**(float(EbN0-10*math.log10(15.0/7.0))/10))

In [11]:
def simulate_ber3(EbN0):
    """ All the work's done here: create flow graph, run, read out BER """
    print "Eb/N0 = {} dB".format(EbN0)
    fg = BERAWGNSimu3(EbN0)
    fg.run()
    res = numpy.sum(fg.sink.data())
    print("BER: {}\n".format(res))
    return res

In [ ]:
ber_simu3  = [simulate_ber3(x) for x in EbN0_range]

Eb/N0 = 0 dB
BER: 0.239314202685

Eb/N0 = 1 dB
BER: 0.179192602634

Eb/N0 = 2 dB


In [ ]:
class BERAWGNSimu4(gr.top_block):
    " This contains the simulation flow graph "
    def __init__(self, EbN0):
        gr.top_block.__init__(self)
        self.const = digital.bpsk_constellation()
        # Source is N_BITS bits, non-repeated

        #print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        #print("arity: ", self.const.arity())

        data = map(int, numpy.random.randint(0, self.const.arity(), int(N_BITS/self.const.bits_per_symbol())))
        src   = blocks.vector_source_b(data, False)
        enc   = BCH_Encoder(15, 2)
        mod   = digital.chunks_to_symbols_bc((self.const.points()), 1)
        add   = blocks.add_vcc()
        diff_enc = digital.diff_encoder_bb(2)
        diff_enc2 = digital.diff_encoder_bb(2)
        diff_dec = digital.diff_decoder_bb(2)
        diff_dec2 = digital.diff_decoder_bb(2)

        #print("Gaussian amplitude: ", self.EbN0_to_noise_voltage(EbN0))

        noise = analog.noise_source_c(analog.GR_GAUSSIAN,
                                      self.EbN0_to_noise_voltage(EbN0),
                                      RAND_SEED)
        demod = digital.constellation_decoder_cb(self.const.base())
        dec   = BCH_Decoder(15, 2)
        ber   = BitErrors(self.const.bits_per_symbol())
        self.sink  = blocks.vector_sink_f()
        self.connect(src, diff_enc, diff_enc2, enc, mod, add, demod, dec, diff_dec, diff_dec2, ber, self.sink)
        self.connect(noise, (add, 1))
        self.connect(src, (ber, 1))

    def EbN0_to_noise_voltage(self, EbN0):
        """ Converts Eb/N0 to a complex noise voltage (assuming unit symbol power) """
        #print("BITS PER SYMBOL: ", self.const.bits_per_symbol())
        return 1.0 / math.sqrt(self.const.bits_per_symbol() * 10**(float(EbN0-10*math.log10(15.0/7.0))/10))

In [ ]:
def simulate_ber4(EbN0):
    """ All the work's done here: create flow graph, run, read out BER """
    print "Eb/N0 = {} dB".format(EbN0)
    fg = BERAWGNSimu4(EbN0)
    fg.run()
    res = numpy.sum(fg.sink.data())
    print("BER: {}\n".format(res))
    return res

In [ ]:
ber_simu4  = [simulate_ber4(x) for x in EbN0_range]

In [ ]:
f = pylab.figure()
s = f.add_subplot(1,1,1)
# s.semilogy(EbN0_range, ber_theory, 'g-.', label="Theoretical DE-BPSK")
#s.semilogy(EbN0_range, ber_theory2, 'y-.', label="Theoretical DE-BPSK + BCH(15, 7, 2)")
s.semilogy(EbN0_range, ber_simu, 'b-*', label="BPSK")
s.semilogy(EbN0_range, ber_simu2, 'r-o', label="BCH(15, 7, 2) + BPSK")
s.semilogy(EbN0_range, ber_simu3, 'g-o', label="DE(1) + BCH(15, 7, 2) + BPSK")
s.semilogy(EbN0_range, ber_simu4, 'k-o', label="DE(2) + BCH(15, 7, 2) + BPSK")

s.set_title('BER Simulation - DE-BPSK')
s.set_xlabel('Eb/N0 (dB)')
s.set_ylabel('BER')
s.legend()
s.grid()
# pylab.savefig("plot-bch-15-7-2.pdf", dpi = 300)
pylab.show()